# Round Trip Tearsheet

When evaluating the performance of an investing strategy, it is helpful to quantify the frequency, duration, and profitability of its independent bets, or "round trip" trades. A round trip trade is started when a new long or short position is opened and is only completed when the number of shares in that position returns to or crosses zero. 

The intent of the round trip tearsheet is to help differentiate strategies that profited off a few lucky trades from strategies that profited repeatedly from genuine alpha. Breaking down round trip profitability by traded name and sector can also help inform universe selection and identify exposure risks. For example, even if your equity curve looks robust, if only two securities in your universe of fifteen names contributed to overall profitability, you may have reason to question the logic of your strategy.

To identify round trips, pyfolio groups transactions by symbol and identifies the points at which each position amount leaves and returns to zero. Behind the scenes, transactions that cause position amounts to flip directly from long to short or short to long are divided into separate transactions so that distinct round trips can be identified. In calculating round trips, pyfolio will also append position closing transactions at the last timestamp in the positions data. This closing transaction will cause the PnL from any open positions to realized as completed round trips.

**Note: The round trip method of quantifying performance is not applicable to every style of strategy.** For instance, simple rebalancing algorithms make very few, if any, round trip trades. The results of the round trip tearsheet will be less informative for any strategy that doesn't entirely exit positions.

In [1]:
%matplotlib inline
import gzip
import os
import pandas as pd
import pyfolio as pf

In [2]:
transactions = pd.read_csv(gzip.open('../tests/test_data/test_txn.csv.gz'),
                    index_col=0, parse_dates=0)
positions = pd.read_csv(gzip.open('../tests/test_data/test_pos.csv.gz'),
                    index_col=0, parse_dates=0)
returns = pd.read_csv(gzip.open('../tests/test_data/test_returns.csv.gz'),
                    index_col=0, parse_dates=0)

In [3]:
# Optional: Sector mappings may be passed in as a dict or pd.Series. If a mapping is
# provided, PnL from symbols with mappings will be summed to display profitability by sector.
sect_map = {'COST': 'Consumer Goods', 'INTC':'Technology', 'CERN':'Healthcare', 'GPS':'Technology',
            'MMM': 'Construction', 'DELL': 'Technology', 'AMD':'Technology'}

In [5]:
rts = pf.round_trips.extract_round_trips(transactions, portfolio_value=positions.sum(axis='columns'))

In [6]:
rts.head()

,close_dt,duration,long,open_dt,pnl,rt_returns,symbol,returns
0,2004-01-13,1 days,True,2004-01-12,-208.800000,-0.036779,AMD,-0.002110
1,2004-01-16,1 days,True,2004-01-15,35.260000,0.054088,AMD,0.000349
2,2004-01-20,8 days,True,2004-01-12,1637.659065,0.112071,AMD,0.016318
3,2004-01-21,1 days,False,2004-01-20,287.119806,0.085155,AMD,0.002866
4,2004-01-22,2 days,False,2004-01-20,103.349947,0.112198,AMD,0.001038


In [7]:
pf.round_trips.print_round_trip_stats(rts)

Summary stats,All round_trips,Long round_trips,Short round_trips
Total number of round_trips,5819.00,4664.00,1155.00
Percent profitable,0.55,0.55,0.52
Winning round_trips,3180.00,2584.00,596.00
Losing round_trips,2613.00,2060.00,553.00
Even round_trips,26.00,20.00,6.00


PnL stats,All round_trips,Long round_trips,Short round_trips
Total profit,$66840.00,$63308.68,$3531.32
Gross profit,$402341.03,$381761.36,$20579.67
Gross loss,$-335501.03,$-318452.68,$-17048.35
Profit factor,$1.20,$1.20,$1.21
Avg. trade net profit,$11.49,$13.57,$3.06
Avg. winning trade,$126.52,$147.74,$34.53
Avg. losing trade,$-128.40,$-154.59,$-30.83
Ratio Avg. Win:Avg. Loss,$0.99,$0.96,$1.12
Largest winning trade,$9977.28,$9977.28,$1623.24
Largest losing trade,$-18557.39,$-18557.39,$-661.29


Duration stats,All round_trips,Long round_trips,Short round_trips
Avg duration,7 days 18:01:03.103626,9 days 01:33:23.773584,2 days 10:39:35.064935
Median duration,3 days 00:00:00,4 days 00:00:00,2 days 00:00:00
Avg # round_trips per day,20.42,16.36,96.25
Avg # round_trips per month,428.77,343.66,2021.25


Return stats,All round_trips,Long round_trips,Short round_trips
Avg returns all round_trips,0.01%,0.01%,0.00%
Avg returns winning,0.10%,0.12%,0.03%
Avg returns losing,-0.11%,-0.13%,-0.03%
Median returns all round_trips,0.00%,0.00%,0.00%
Median returns winning,0.01%,0.02%,0.01%
Median returns losing,-0.01%,-0.01%,-0.00%
Largest winning trade,7.08%,7.08%,1.33%
Largest losing trade,-16.77%,-16.77%,-0.72%


Symbol stats,AMD,CERN,COST,DELL,GPS,INTC,MMM
Avg returns all round_trips,-0.35%,1.80%,2.46%,-2.93%,-0.06%,1.91%,1.04%
Avg returns winning,16.52%,12.64%,8.21%,8.66%,7.86%,8.46%,8.13%
Avg returns losing,-18.75%,-12.22%,-5.54%,-15.03%,-9.46%,-5.70%,-9.17%
Median returns all round_trips,0.02%,0.06%,0.10%,0.00%,0.03%,0.02%,0.08%
Median returns winning,1.79%,1.68%,1.23%,1.25%,1.18%,0.96%,1.12%
Median returns losing,-1.28%,-0.94%,-0.71%,-0.86%,-0.79%,-0.50%,-0.64%
Largest winning trade,708.48%,605.28%,228.45%,270.27%,195.77%,248.91%,254.67%
Largest losing trade,-1477.84%,-544.99%,-415.15%,-1676.90%,-348.04%,-511.02%,-339.08%
